In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Colab Notebooks/nn hw3"

/content/drive/.shortcut-targets-by-id/16pPvnYNuR5IKHD3x4aWZ2qNj3oNEGnCa/nn hw3


In [12]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import time
import random
import numpy as np
import pandas as pd
import tqdm
import pickle

In [4]:
# !wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Magazine_Subscriptions.json.gz

--2023-06-13 17:56:04--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Magazine_Subscriptions.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12875273 (12M) [application/x-gzip]
Saving to: ‘Magazine_Subscriptions.json.gz.2’

Magazine_Subscripti 100%[===================>]  12.28M  7.26MB/s    in 1.7s    

2023-06-13 17:56:07 (7.26 MB/s) - ‘Magazine_Subscriptions.json.gz.2’ saved [12875273/12875273]



where

- `reviewerID` - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- `asin` - ID of the product, e.g. 0000013714
- `reviewerName` - name of the reviewer
- `vote` - helpful votes of the review
- `style` - a disctionary of the product metadata, e.g., "Format" is "Hardcover"
- `reviewText` - text of the review
- `overall` - rating of the product
- `summary` - summary of the review
- `unixReviewTime` - time of the review (unix time)
- `reviewTime` - time of the review (raw)
- `image` - images that users post after they have received the product

In [5]:
def parse(path):
  g = gzip.open(path, 'rb')
  indx = 0
  for l in g:
    yield json.loads(l)


def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Magazine_Subscriptions.json.gz')
df = df[['overall', 'reviewText']]
df['overall'] = df['overall'].astype(np.int32) - 1
df.dropna(inplace=True)
df = df[list(df.columns)[::-1]]
df.to_csv('amazon.csv', index = False)
df

,reviewText,overall
0,"for computer enthusiast, MaxPC is a welcome si...",4
1,Thank god this is not a Ziff Davis publication...,4
2,Antiques Magazine is a publication made for an...,2
3,This beautiful magazine is in itself a work of...,4
4,A great read every issue.,4
...,...,...
89684,This was a nice surprise for my boyfriend. He ...,4
89685,Magazine looks like it is printed on recycled ...,0
89686,cant go wrong with an SI subscription\nvery pl...,4
89687,This magazine is by far my all time favorite o...,4


In [6]:
del df

In [7]:
import torch
import torch.nn.functional as F
import torchtext
from torchdata.datapipes.iter import IterableWrapper, FileOpener
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Sampler, Dataset

torch.backends.cudnn.deterministic = True

In [8]:
class CustomBatchSamplerSimilarLength(Sampler):
    def __init__(self, dataset, batch_size, tokenizer, indices=None, shuffle=True):
        self.batch_size = batch_size
        self.shuffle = shuffle
        # get the indices and length
        self.indices = [(i, len(tokenizer(s[0]))) for i, s in enumerate(dataset)]
        # if indices are passed, then use only the ones passed (for ddp)
        if indices is not None:
            self.indices = torch.tensor(self.indices)[indices].tolist()

    def __iter__(self):
        if self.shuffle:
            random.shuffle(self.indices)

        pooled_indices = []
        # create pool of indices with similar lengths
        for i in range(0, len(self.indices), self.batch_size * 100):
          pooled_indices.extend(sorted(self.indices[i:i + self.batch_size * 100], key=lambda x: x[1]))
        self.pooled_indices = [x[0] for x in pooled_indices]

        # Comment in for validation
        #self.pooled_lengths = [x[1] for x in pooled_indices]
        #print(self.pooled_lengths)
        #print(self.pooled_indices)

        # yield indices for current batch
        batches = [self.pooled_indices[i:i + self.batch_size] for i in
                   range(0, len(self.pooled_indices), self.batch_size)]

        if self.shuffle:
            random.shuffle(batches)
        for batch in batches:
            yield batch

    def __len__(self):
        return len(self.pooled_indices) // self.batch_size

In [14]:
def mkdir(directory: str):
    if not os.path.exists(directory):
        os.makedirs(directory)

import matplotlib.pyplot as plt


def custom_plot_training_stats(acc_hist, loss_hist, phase_list, title: str, dir: str):
    fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize=[14, 6], dpi=100)

    for phase in phase_list:
        lowest_loss_x = np.argmin(np.array(loss_hist[phase]))
        lowest_loss_y = loss_hist[phase][lowest_loss_x]

        ax1.annotate("{:.4f}".format(lowest_loss_y), [lowest_loss_x, lowest_loss_y])
        ax1.plot(loss_hist[phase], '-x', label=f'{phase} loss', markevery = [lowest_loss_x])

        ax1.set_xlabel(xlabel='epochs')
        ax1.set_ylabel(ylabel='loss')

        ax1.grid(color = 'green', linestyle = '--', linewidth = 0.5, alpha=0.75)
        ax1.legend()
        ax1.label_outer()

    # acc:
    for phase in phase_list:
        highest_acc_x = np.argmax(np.array(acc_hist[phase]))
        highest_acc_y = acc_hist[phase][highest_acc_x]

        ax2.annotate("{:.4f}".format(highest_acc_y), [highest_acc_x, highest_acc_y])
        ax2.plot(acc_hist[phase], '-x', label=f'{phase} loss', markevery = [highest_acc_x])

        ax2.set_xlabel(xlabel='epochs')
        ax2.set_ylabel(ylabel='acc')

        ax2.grid(color = 'green', linestyle = '--', linewidth = 0.5, alpha=0.75)
        ax2.legend()
        #ax2.label_outer()

    fig.suptitle(f'{title}')

    mkdir(dir)
    plt.savefig(f'{dir}/{title}_ acc loss.jpg')
    plt.clf()

In [30]:
class LSTM(torch.nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)

        self.rnn = torch.nn.LSTM(input_size=embedding_dim,
                                 hidden_size=hidden_dim,
                                 num_layers=num_layers
                                 )

        self.fc = torch.nn.Linear(hidden_dim, output_dim)


    def forward(self, text):
        # text dim: [sentence length, batch size]
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]

        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [num_layers, batch size, hidden dim]

        ### Changed it
        ### hidden.squeeze_(0)
        # hidden_final = hidden[-1]
        # hidden_final: [batch size, hidden dim]
        # output = self.fc(hidden_final)

        clf_output = output[-1] # [batch size, hidden dim]
        output = self.fc(clf_output)

        return output

class RNN(torch.nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)

        self.rnn = torch.nn.RNN(input_size=embedding_dim,
                                 hidden_size=hidden_dim,
                                 num_layers=num_layers,
                                 nonlinearity='relu'
                                 )

        self.fc = torch.nn.Linear(hidden_dim, output_dim)


    def forward(self, text):
        # text dim: [sentence length, batch size]
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]

        output, hidden = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [num_layers, batch size, hidden dim]

        clf_output = output[-1] # [batch size, hidden dim]
        output = self.fc(clf_output)

        return output

class GRU(torch.nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)

        # tru dropout
        self.gru = torch.nn.GRU(input_size=embedding_dim,
                                 hidden_size=hidden_dim,
                                 num_layers=num_layers,
                                 )

        self.fc = torch.nn.Linear(hidden_dim, output_dim)


    def forward(self, text):
        # text dim: [sentence length, batch size]
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]

        output, hidden = self.gru(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [num_layers, batch size, hidden dim]

        clf_output = output[-1] # [batch size, hidden dim]
        output = self.fc(clf_output)

        return output



In [31]:
class sentiment_clf():
    def __init__(self, device, rand_seed = 123):
        self.RANDOM_SEED = rand_seed
        torch.manual_seed(self.RANDOM_SEED)
        self.device = device

    def load_data(self, csv_name: str, debug=False):
        self.csv_name = csv_name
        self.datapipe = IterableWrapper([f"{csv_name}"])
        self.datapipe = FileOpener(self.datapipe, mode='b')
        self.datapipe = self.datapipe.parse_csv(skip_lines=1) # skip_lines=1 skip headers

        if debug:
            indx = 0
            for sample in self.datapipe:
                print(sample)
                indx += 1
                if indx > 6:
                    break

        N_ROWS = len(list(self.datapipe))
        self.train_dp, self.valid_dp, self.test_dp = self.datapipe.random_split(total_length=N_ROWS, weights={"train": 0.8, "valid": 0.1, "test": 0.1}, seed=self.RANDOM_SEED)
        if debug:
            print('N_ROWS:', N_ROWS)
            print(f'Num Train: {len(list(self.train_dp))}')
            print(f'Num Valid: {len(list(self.valid_dp))}')
            print(f'Num Test: {len(list(self.test_dp))}')


    def build_vocab(self, vocab_size, tokenizer_name: str = 'spacy', save_vocab=True, debug=False):
        self.vocab_size = vocab_size
        self.tokenizer_name = tokenizer_name

        if tokenizer_name == 'spacy':
            lang = 'en_core_web_sm'
        else:
            lang = 'en'
        self.tokenizer = get_tokenizer(tokenizer=tokenizer_name, language=lang)

        def yield_tokens(data_iter):
            for text, _ in data_iter:
                yield self.tokenizer(text)

        self.vocab = build_vocab_from_iterator(yield_tokens(self.train_dp), specials=["<UNK>", "<PAD>"], max_tokens=self.vocab_size)
        self.vocab.set_default_index(self.vocab["<UNK>"])
        if debug:
            print("Vocabulary size: ", len(self.vocab))
            print(self.vocab.get_itos()[:10]) # itos = integer-to-string
            print(f"Default index: {self.vocab.get_default_index()}")

        self.padding_value=self.vocab['<PAD>']

        self.text_transform = lambda x: [self.vocab[token] for token in self.tokenizer(x)]
        self.label_transform = lambda x: int(x)

        if debug:
            print(f"the: {self.vocab['the']}")
            # And an indirect way, using get_stoi() to get a dictionary of tokens and values
            print(f"the: {self.vocab.get_stoi()['the']}") # stoi = string-to-integer
            # What is the padding value?
            print(f"<PAD>: {self.vocab['<PAD>']}")

            # Print out the output of text_transform
            print("input to the text_transform:", "here is an example")
            print("output of the text_transform:", self.text_transform("here is an example"))
            for text, label in self.train_dp:
                print(label)
                print(text)
                first_text_tokens = self.text_transform(text)
                print(self.text_transform(text))
                print(self.label_transform(label))
                break

            vocab_itos = self.vocab.get_itos()
            vec_vocab_itos = np.vectorize(lambda x: vocab_itos[x])
            print(vec_vocab_itos(first_text_tokens))

    def create_dataloaders(self, batch_size, debug=False):
        self.batch_size = batch_size

        def collate_batch(batch):
            text_list, label_list = [], []
            for (_text, _label) in batch:
                processed_text = torch.tensor(self.text_transform(_text))
                text_list.append(processed_text)
                label_list.append(self.label_transform(_label))
            return pad_sequence(text_list, padding_value=self.padding_value).to(self.device), torch.tensor(label_list).to(self.device)


        train_dp_list = list(self.train_dp)
        valid_dp_list = list(self.valid_dp)
        test_dp_list = list(self.test_dp)

        self.train_loader = DataLoader(train_dp_list,
                                batch_sampler=CustomBatchSamplerSimilarLength(dataset = train_dp_list,
                                                                        batch_size=self.batch_size,
                                                                              tokenizer=self.tokenizer,
                                                                              shuffle=True),
                                collate_fn=collate_batch)
        self.valid_loader = DataLoader(valid_dp_list,
                                batch_sampler=CustomBatchSamplerSimilarLength(dataset = valid_dp_list,
                                                                        batch_size=self.batch_size,
                                                                        tokenizer=self.tokenizer,
                                                                        shuffle=False),
                                collate_fn=collate_batch)
        self.test_loader = DataLoader(test_dp_list,
                                batch_sampler=CustomBatchSamplerSimilarLength(dataset = test_dp_list,
                                                                        batch_size=self.batch_size,
                                                                        tokenizer=self.tokenizer,
                                                                        shuffle=False),
                                collate_fn=collate_batch)

        if debug:
            print('Train')
            count = 20
            for text_batch, label_batch in self.train_loader:
                print(f'Text matrix size: {text_batch.size()}')
                print(f'Target vector size: {label_batch.size()}')
                count -= 1
                if count <= 0:
                    count = 5
                    break

            print('\nValid:')
            for text_batch, label_batch in self.valid_loader:
                print(f'Text matrix size: {text_batch.size()}')
                print(f'Target vector size: {label_batch.size()}')
                count -= 1
                if count <= 0:
                    count = 5
                    break

            print('\nTest:')
            for text_batch, label_batch in self.test_loader:
                print(f'Text matrix size: {text_batch.size()}')
                print(f'Target vector size: {label_batch.size()}')
                count -= 1
                if count <= 0:
                    count = 5
                    break

            text_batch, label_batch = next(iter(self.train_loader))
            print(f'Text matrix size: {text_batch.size()}')
            print(f'Target vector size: {label_batch.size()}')

            text_batch, label_batch = next(iter(self.train_loader))
            print(f'Text matrix size: {text_batch.size()}')
            print(f'Target vector size: {label_batch.size()}')


    def compute_accuracy_loss(self, model, data_loader):
        model.eval()
        running_loss = 0.0
        with torch.no_grad():

            correct_pred, num_examples = 0, 0
            for i, (features, targets) in enumerate(data_loader):

                features = features.to(self.device)
                targets = targets.to(self.device)

                logits = model(features)
                loss = torch.nn.functional.cross_entropy(logits, targets)
                running_loss += loss.item() * targets.size(0)

                _, predicted_labels = torch.max(logits, 1)

                num_examples += targets.size(0)
                correct_pred += (predicted_labels == targets.float()).sum().to("cpu")

        return float(correct_pred)/num_examples * 100, running_loss / num_examples

    def build_model(self, model_name: str,
                    embedding_dim: int,
                    hidden_dim: int,
                    num_layers: int,
                    num_classes: int,
                    debug=False):
        if model_name == 'lstm':
            self.model = LSTM(input_dim=len(self.vocab),
                              embedding_dim=embedding_dim,
                              hidden_dim=hidden_dim,
                              num_layers=num_layers,
                              output_dim=num_classes
                              )
        elif model_name == 'rnn':
            self.model = RNN(input_dim=len(self.vocab),
                              embedding_dim=embedding_dim,
                              hidden_dim=hidden_dim,
                              num_layers=num_layers,
                              output_dim=num_classes
                              )

        elif model_name == 'gru':
            self.model = GRU(input_dim=len(self.vocab),
                              embedding_dim=embedding_dim,
                              hidden_dim=hidden_dim,
                              num_layers=num_layers,
                              output_dim=num_classes
                              )


        self.model_name = model_name
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        if debug:
            print(self.model)

    def train_model(self, lr: float,
                    num_epochs,
                    scheduler_on='valid_acc'):
        self.lr = lr
        self.num_epochs =num_epochs

        self.model = self.model.to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer,
                                                                    factor=0.1,
                                                                    mode='max',
                                                                    verbose=True)
        self.acc_history = {'train': [], 'valid': []}
        self.loss_history = {'train': [], 'valid': []}

        start_time = time.time()
        minibatch_loss_list = []
        # train_acc_list, valid_acc_list = [], []

        for epoch in range(self.num_epochs):

            self.model.train()
            for batch_idx, (features, targets) in enumerate(self.train_loader):

                features = features.to(self.device)
                targets = targets.to(self.device)

                # ## FORWARD AND BACK PROP
                logits = self.model(features)
                loss = torch.nn.functional.cross_entropy(logits, targets)
                self.optimizer.zero_grad()

                loss.backward()

                # ## UPDATE MODEL PARAMETERS
                self.optimizer.step()

                # ## LOGGING
                minibatch_loss_list.append(loss.item())
                # if not batch_idx % logging_interval:
                #     print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                #         f'| Batch {batch_idx:04d}/{len(self.train_loader):04d} '
                #         f'| Loss: {loss:.4f}')

            with torch.no_grad():  # save memory during inference
                train_acc, train_loss = self.compute_accuracy_loss(self.model, self.train_loader)
                valid_acc, valid_loss = self.compute_accuracy_loss(self.model, self.valid_loader)
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                    f'| Train: {train_acc :.2f}% '
                    f'| Validation: {valid_acc :.2f}%',
                    f'| Train Loss: {train_loss :.4f} '
                    f'| Validation Loss: {valid_loss :.4f}',
                      )

                self.acc_history['train'].append(train_acc)
                self.acc_history['valid'].append(valid_acc)
                self.loss_history['train'].append(train_loss)
                self.loss_history['valid'].append(valid_loss)


            elapsed = (time.time() - start_time)/60
            print(f'Time elapsed: {elapsed:.2f} min')

            if self.scheduler is not None:

                if scheduler_on == 'valid_acc':
                    self.scheduler.step(self.acc_history['valid'][-1])
                elif scheduler_on == 'minibatch_loss':
                    self.scheduler.step(minibatch_loss_list[-1])
                else:
                    raise ValueError(f'Invalid `scheduler_on` choice.')

        elapsed = (time.time() - start_time)/60
        print(f'Total Training Time: {elapsed:.2f} min')

        self.test_acc, self.test_loss = self.compute_accuracy_loss(self.model, self.test_loader)
        print(f'Test accuracy {self.test_acc :.2f}%')

        # return minibatch_loss_list, train_acc_list, valid_acc_list

    def save_results(self, plot=False):

        self.exp_name = f'model_{self.model_name}-voacb_{self.vocab_size}-embed_{self.embedding_dim}-hidd_{self.hidden_dim}-num_lyr_{self.num_layers}-bs_{self.batch_size}-lr_{self.lr}-epochs_{self.num_epochs}'

        mkdir('stats')
        with open(f'stats/{self.exp_name}-stats.pickle', 'wb') as handle:
            pickle.dump({
                'exp_name': self.exp_name,
                'model_name': self.model_name,
                'vocab_size': self.vocab_size,
                'embedding_dim': self.embedding_dim,
                'hidden_dim': self.hidden_dim,
                'num_layers': self.num_layers,
                'batch_size': self.batch_size,
                'num_epochs': self.num_epochs,
                'lr': self.lr,
                'acc_history': self.acc_history,
                'loss_history': self.loss_history,
                'test_acc': self.test_acc,
                'test_loss': self.test_loss
            }, handle)


        custom_plot_training_stats(self.acc_history, self.loss_history, ['train', 'valid'], title=self.exp_name, dir='images')

In [41]:
RANDOM_SEED = 123

VOCABULARY_SIZE = 40000 # 40000
LEARNING_RATE = 0.001 # 0.005
BATCH_SIZE = 64
NUM_EPOCHS = 5
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

NUM_LAYERS = 3
EMBEDDING_DIM = 128 # 128
HIDDEN_DIM = 256 # 256
NUM_CLASSES = 5


rnn_clf = sentiment_clf(DEVICE)
db = True
rnn_clf.load_data('amazon.csv', debug=db)
# rnn_clf.build_vocab(VOCABULARY_SIZE, tokenizer_name='basic_english', debug=db)
rnn_clf.build_vocab(VOCABULARY_SIZE, debug=db)
rnn_clf.create_dataloaders(BATCH_SIZE, debug=db)
rnn_clf.build_model('gru', embedding_dim=EMBEDDING_DIM,
                    hidden_dim=HIDDEN_DIM, num_layers=NUM_LAYERS,
                    num_classes=NUM_CLASSES, debug=db)

rnn_clf.train_model(
    lr=LEARNING_RATE,
    num_epochs=NUM_EPOCHS)
rnn_clf.save_results()

['for computer enthusiast, MaxPC is a welcome sight in your mailbox. i can remember for years savorying every page of "boot" (as it was called in beginning) as i was (and still am) obcessed with PC\'s. Anyone, from advanced users - to beginners looking for knowledge - can profit from every issue of MaxPC. the icing on the cake is the subscription that comes with a CD-ROM as it is packed with demos, utilities, and other useful apps (very helpful for those not blessed with broadband connections). Until I discovered the community of hardware enthusiast web sites, MaxPC, formerly "boot", was my only really informative source for computing news and articles. To this day, i consider my subscription to it worth more than 10 subscriptions to most other computing mags. I can\'t wait until they merge with DVD media and maybe end up offering more info on Divx codecs, encoding your own movies, and best bang for the buck audio and video equipment. Try a few issues (with CD)and you may get hooked...

<Figure size 1400x600 with 0 Axes>

<Figure size 1400x600 with 0 Axes>

In [48]:

for _, _, files in os.walk("stats"):
    temp = files

In [71]:
dict_res = {'model_name': [], 'test_acc': []}

for pkl_file in temp:
    with open(f'stats/{pkl_file}', "rb") as f:
        data = pickle.load(f)
        model_name = data['model_name']
        num_layer = data['num_layers']
        dict_res['model_name'].append(f'{model_name}<{num_layer}>')
        dict_res['test_acc'].append(data['test_acc'])

        # print(data['test_acc'], data['exp_name'])

In [62]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
from tabulate import tabulate

In [72]:
print(tabulate(dict_res, headers="keys", tablefmt="latex"))

\begin{tabular}{lr}
\hline
 model\_name   &   test\_acc \\
\hline
 lstm\ensuremath{<}2\ensuremath{>}      &    70.6447 \\
 gru\ensuremath{<}2\ensuremath{>}       &    71.1912 \\
 rnn\ensuremath{<}2\ensuremath{>}       &    61.3875 \\
 rnn\ensuremath{<}1\ensuremath{>}       &    59.8706 \\
 gru\ensuremath{<}1\ensuremath{>}       &    71.3362 \\
 lstm\ensuremath{<}1\ensuremath{>}      &    70.6893 \\
 lstm\ensuremath{<}3\ensuremath{>}      &    69.7078 \\
 rnn\ensuremath{<}3\ensuremath{>}       &    11.6551 \\
 rnn\ensuremath{<}3\ensuremath{>}       &    65.0457 \\
 gru\ensuremath{<}3\ensuremath{>}       &    70.9458 \\
\hline
\end{tabular}
